In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
from pathlib import Path
from sklearn import datasets, metrics, model_selection, svm
from sklearn.utils import resample
%matplotlib inline

In [6]:
data = pd.read_csv(r"C:\Users\sshin\Desktop\SYSHIN0228\RERC_STARTS_Sensor_Analysis\Machine_Learning_6MWT\Feature_Matrix_Prediction_6MWT_BBS.csv", sep=',')
data = data.dropna()

CM = np.zeros((3,3))
print(CM)

for n in [60]:
    data_filter = data.query('cut_off_time == @n')
#     print(data_filter)

    df_majority = data_filter[data_filter.Ambul=='Mild']
    df_minority = data_filter[data_filter.Ambul=='Severe']
    df_minority2 = data_filter[data_filter.Ambul=='Moderate']
    
    df_minority_upsampled = resample(df_minority, replace=True, n_samples=26, random_state=123)
    df_minority2_upsampled = resample(df_minority2, replace=True, n_samples=26, random_state=123)
    data_filter = pd.concat([df_majority, df_minority_upsampled, df_minority2_upsampled])
    

    X = data_filter[[
        'Age', 'SexM0F1', 'Height', 'Weight',
        'FIM_AD', 'BBS_AD', 'MWT10_SSV_AD', 'MWT6_AD', 'TUG_AD',
#         'AoM_Pel_tilt', 'AoM_Pel_oblq', 'AoM_Pel_ro', 'AoM_Pel_norm',
#         'AoM_Ankle_US_x', 'AoM_Ankle_US_y', 'AoM_Ankle_US_z', 'AoM_Ankle_US_norm',
#         'AoM_Ankle_AS_x', 'AoM_Ankle_AS_y', 'AoM_Ankle_AS_z', 'AoM_Ankle_AS_norm',
#         'f50_ML', 'f50_AP', 'f95_ML', 'f95_AP', 'spectral_centroid_AP', 'spectral_centroid_ML',
#         'max_accAP', 'max_accML', 'mean_accAP', 'mean_accAP', 'rms_AP', 'rms_ML', 'jerk_AP', 'jerk_ML',
#         'mean_velAP', 'mean_velML', 'length_swayAPAcc', 'length_swayMLAcc',
#         'SC_Gyr_norm_mean', 'DLS_R_Gyr_norm_mean', 'DLS_L_Gyr_norm_mean',
#         'SC_Gyr_norm_range', 'DLS_R_Gyr_norm_range', 'DLS_L_Gyr_norm_range',
#         'SC_Gyr_norm_rms', 'DLS_R_Gyr_norm_rms', 'DLS_L_Gyr_norm_rms',
#         'SC_Gyr_norm_std', 'DLS_R_Gyr_norm_std', 'DLS_L_Gyr_norm_std',
#         'SC_Gyr_norm_skew', 'DLS_R_Gyr_norm_skew', 'DLS_L_Gyr_norm_skew',
#         'SC_Gyr_norm_kurtosis', 'DLS_R_Gyr_norm_kurtosis', 'DLS_L_Gyr_norm_kurtosis',
#         'SC_Gyr_norm_SamEn', 'DLS_R_Gyr_norm_SamEn', 'DLS_L_Gyr_norm_SamEn',
#         'SC_Acc_norm_mean', 'DLS_R_Acc_norm_mean', 'DLS_L_Acc_norm_mean',
#         'SC_Acc_norm_range', 'DLS_R_Acc_norm_range', 'DLS_L_Acc_norm_range',
#         'SC_Acc_norm_rms', 'DLS_R_Acc_norm_rms', 'DLS_L_Acc_norm_rms',
#         'SC_Acc_norm_std', 'DLS_R_Acc_norm_std', 'DLS_L_Acc_norm_std',
#         'SC_Acc_norm_skew', 'DLS_R_Acc_norm_skew', 'DLS_L_Acc_norm_skew',
#         'SC_Acc_norm_kurtosis', 'DLS_R_Acc_norm_kurtosis', 'DLS_L_Acc_norm_kurtosis',
#         'SC_Acc_norm_SamEn', 'DLS_R_Acc_norm_SamEn', 'DLS_L_Acc_norm_SamEn'
                ]].values
    
    print(data_filter['Ambul'])
    
    factor = pd.factorize(data_filter['Ambul'])
    data_filter.Ambul = factor[0]
    definitions = factor[1]
    print(data_filter.Ambul.head())
    print(definitions) 
    
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scale = min_max_scaler.fit_transform(X)
    y = data_filter['Ambul'].values


    print(X_scale)
    print(y)
    
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    i = 0

    y_test_ = np.zeros((len(y)))
    y_pred_ = np.zeros((len(y)))
    y_probs_ = np.zeros((len(y),3))
    
    correct = 0
    incorrect = 0
    for train_index, test_index in loo.split(X):
#         print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_scale[train_index], X_scale[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf = RandomForestClassifier(max_depth=70, random_state=0)
        clf.fit(X_train,y_train)
        
        y_pred = clf.predict(X_test)
        
        y_test_[i] = y_test
        y_pred_[i] = y_pred
        y_probs_[i] = clf.predict_proba(X_test)
        
        reversefactor = dict(zip(range(3),definitions))
        y_test = np.vectorize(reversefactor.get)(y_test)
        y_pred = np.vectorize(reversefactor.get)(y_pred)    
         
        print(y_test) 
        print(y_pred)

        
        if y_test == ['Mild'] and y_pred == ['Mild']:
            CM[0][0] += 1
        elif y_test == ['Mild'] and y_pred == ['Severe']:
            CM[0][1] += 1
        elif y_test == ['Mild'] and y_pred == ['Moderate']:
            CM[0][2] += 1
        elif y_test == ['Severe'] and y_pred == ['Mild']:
            CM[1][0] += 1
        elif y_test == ['Severe'] and y_pred == ['Severe']:
            CM[1][1] += 1
        elif y_test == ['Severe'] and y_pred == ['Moderate']:
            CM[1][2] += 1
        elif y_test == ['Moderate'] and y_pred == ['Mild']:
            CM[2][0] += 1
        elif y_test == ['Moderate'] and y_pred == ['Severe']:
            CM[2][1] += 1
        elif y_test == ['Moderate'] and y_pred == ['Moderate']:
            CM[2][2] += 1
            
        print(CM)
        
        if y_test == y_pred:
            correct = correct + 1
        else:
            incorrect = incorrect + 1

        i = i+1
        print("Iteration:", i)  

    print("Correct: ", correct)
    print("Incorrect: ", incorrect)
    
target_names = ['Moderate', 'Severe', 'Moderate']    
print(confusion_matrix(y_test_,y_pred_))
print(classification_report(y_test_,y_pred_, target_names = target_names))




[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
221        Mild
223        Mild
224        Mild
225        Mild
228        Mild
         ...   
233    Moderate
256    Moderate
238    Moderate
256    Moderate
247    Moderate
Name: Ambul, Length: 78, dtype: object
221    0
223    0
224    0
225    0
228    0
Name: Ambul, dtype: int64
Index(['Mild', 'Severe', 'Moderate'], dtype='object')
[[6.89655172e-01 1.00000000e+00 2.49507874e-01 2.43027888e-01
  8.03030303e-01 6.73076923e-01 2.73941068e-01 4.18282434e-01
  8.58091999e-02]
 [1.00000000e+00 0.00000000e+00 6.87500000e-01 7.88844622e-01
  1.81818182e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [4.48275862e-01 1.00000000e+00 2.50000000e-01 8.02124834e-01
  4.54545455e-01 5.76923077e-02 0.00000000e+00 4.98158978e-02
  0.00000000e+00]
 [9.31034483e-01 0.00000000e+00 8.50000000e-01 7.31739708e-01
  3.78787879e-01 2.50000000e-01 3.60497238e-01 2.70673598e-01
  1.51056998e-01]
 [6.72413793e-01 1.00000000e+00 6.18602362e-01 8.618857

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

['Mild']
['Mild']
[[20.  3.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Iteration: 24
['Mild']
['Mild']
[[21.  3.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Iteration: 25
['Mild']
['Mild']
[[22.  3.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Iteration: 26
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
Iteration: 27
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  2.  0.]
 [ 0.  0.  0.]]
Iteration: 28
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  3.  0.]
 [ 0.  0.  0.]]
Iteration: 29
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  4.  0.]
 [ 0.  0.  0.]]
Iteration: 30
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  5.  0.]
 [ 0.  0.  0.]]
Iteration: 31
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  6.  0.]
 [ 0.  0.  0.]]
Iteration: 32
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  7.  0.]
 [ 0.  0.  0.]]
Iteration: 33
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  8.  0.]
 [ 0.  0.  0.]]
Iteration: 34
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0.  9.  0.]
 [ 0.  0.  0.]]
Iteration: 35
['Severe']
['Severe']
[[22. 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

['Severe']
['Severe']
[[22.  3.  1.]
 [ 0. 24.  0.]
 [ 0.  0.  0.]]
Iteration: 50
['Severe']
['Moderate']
[[22.  3.  1.]
 [ 0. 24.  1.]
 [ 0.  0.  0.]]
Iteration: 51
['Severe']
['Severe']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  0.]]
Iteration: 52
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  1.]]
Iteration: 53
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  2.]]
Iteration: 54
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  3.]]
Iteration: 55
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  4.]]
Iteration: 56
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  5.]]
Iteration: 57
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  6.]]
Iteration: 58
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  7.]]
Iteration: 59
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0.  8.]]
Iteration: 60
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0. 23.]]
Iteration: 75
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0. 24.]]
Iteration: 76
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0. 25.]]
Iteration: 77
['Moderate']
['Moderate']
[[22.  3.  1.]
 [ 0. 25.  1.]
 [ 0.  0. 26.]]
Iteration: 78
Correct:  73
Incorrect:  5
[[22  3  1]
 [ 0 25  1]
 [ 0  0 26]]
              precision    recall  f1-score   support

        Mild       1.00      0.85      0.92        26
      Severe       0.89      0.96      0.93        26
    Moderate       0.93      1.00      0.96        26

    accuracy                           0.94        78
   macro avg       0.94      0.94      0.94        78
weighted avg       0.94      0.94      0.94        78



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
